# StoneX API (Trading on Forex.com)

In [1]:
import requests
import os
import datetime
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Account information for the json parameter
account_info =\
    {
        "Password": os.environ.get("password"),
        "UserName": os.environ.get("username"),
        "AppKey": os.environ.get("appkey")
    }

# Header response for the headers parameter
headers =\
    {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

In [3]:
def create_session(method: str, url: str):
    response = requests.request(
    method=method,
    url=url,
    json = account_info,
    headers = headers
    )
    if response.status_code == 200: 
        print(f"[{response.status_code}]: Successful response!") 
        return response.json()
    else:
        print(f"[{response.status_code}]: Invalid response!") 
        return response.status_code

In [4]:
# Contains the session and connection
session_info = create_session(method='POST', url='https://ciapi.cityindex.com/v2/Session')

[200]: Successful response!


In [5]:
# Get more info on the clients trading account
url=f"https://ciapi.cityindex.com/v2/userAccount/ClientAndTradingAccount?UserName={account_info.get("UserName")}&Session={session_info.get('session')}"

# Fetch the json by sending a request
response = requests.get(url=url)

if response.status_code == 200:
    trading_info = response.json()
    print(f"[{response.status_code}]: Additional information on trading account saved!")
else:
    print(f"[{response.status_code}]: Cannot fetch additional information on trading account!")

[200]: Additional information on trading account saved!


In [6]:
# Checks that the session and username combination is valid
auth_response =\
    requests.post(
        url='https://ciapi.cityindex.com/v2/Session/validate',
        json={
            "Session": session_info.get("session"), 
            'UserName': account_info.get('UserName')},
        headers=headers
    )
if auth_response.status_code == 200 and auth_response.json()["isAuthenticated"] == True:
    print("Session and user is valid!")

Session and user is valid!


In [7]:
# Price history for the GBP/USD market
price_bar_response =\
    requests.get(
        url=f'https://ciapi.cityindex.com/TradingAPI/market/401166448/barhistory?interval=DAY&span=1&PriceBars=900&PriceType=BID&UserName={account_info.get("UserName")}&Session={session_info.get('session')}',
    )
price_bars = price_bar_response.json()

# Fix the date values in 'BarDate' in each record of the dictionary
def fix_utc_bardate(value: str):
    value = value.replace('/Date(', '').replace(')/', '')
    value_int = int(value)
    return datetime.datetime.fromtimestamp(timestamp=value_int/1000, tz=datetime.timezone.utc)

for record in price_bars["PriceBars"]:
    date_val = record.get('BarDate')
    record.update({'BarDate': fix_utc_bardate(date_val)})

In [ ]:
# Visualize the data in a candle chart
price_df = pd.DataFrame(data=price_bars["PriceBars"])


# Create a independent figure containing the two
figure = make_subplots(rows=2, cols=1, shared_xaxes=True)

figure.add_trace(go.Candlestick(
        x=price_df["BarDate"],
        open=price_df["Open"].values.flatten(),
        close=price_df["Close"].values.flatten(),
        high=price_df["High"].values.flatten(),
        low=price_df["Low"].values.flatten(),
    ),
    row=1, col=1
)

# Add the volume
figure.add_trace(
    go.Bar(
        x=price_df["BarDate"].values.flatten(),
        y=price_df["Volume"].values.flatten(),
    ),
    row=2, col=1
)

figure.update_layout(title="GBP/USD: Price and Volume Chart", xaxis_rangeslider_visible=False, showlegend=False, height=1200)
figure.show()